#### Install and import Python libraries

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import os
import sys


import matplotlib.pyplot as plt
import numpy as np
import time
import gif
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import DataLoader

from scipy.optimize import linear_sum_assignment as linear_assignment
import os.path as osp

import motmetrics as mm
mm.lap.default_solver = 'lap'
%matplotlib inline


In [ ]:
root_dir = ".."
sys.path.append(os.path.join(root_dir, "src"))


In [ ]:
from mot.data.data_track import MOT16Sequences
from mot.data.data_obj_detect import MOT16ObjDetect
from mot.models.object_detector import FRCNN_FPN
from mot.tracker.advanced import MPNTracker
from mot.eval import evaluate_mot_accums, get_mot_accum, evaluate_obj_detect
from mot.models.gnn import BipartiteNeuralMessagePassingLayer, SimilarityNet
from mot.visualize import plot_sequence, collect_frames_for_gif
from market.models import build_model



## Configuration

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = "cpu"


# Multi-object tracking

## Configuration

In [ ]:
seed = 12345
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True


## load pretrained

In [ ]:
# Define our model, and init
similarity_net = SimilarityNet(
    reid_network=None,  # Not needed since we work with precomputed features
    node_dim=32,
    edge_dim=64,
    reid_dim=512,
    edges_in_dim=6,
    num_steps=10,
).to(device)


In [ ]:
best_ckpt = torch.load(
    osp.join(root_dir, "models", "best_ckpt.pth"),
    map_location=lambda storage, loc: storage,
)
similarity_net.load_state_dict(best_ckpt)
MAX_PATIENCE = 20

tracker = MPNTracker(
    similarity_net=similarity_net.eval(),
    obj_detect=None,
    reid_model=None,
    device=device,
    patience=MAX_PATIENCE,
)


In [ ]:
val_sequences = MOT16Sequences(
    "MOT16-val2", osp.join(root_dir, "data/MOT16"), vis_threshold=0.0
)
train_db = torch.load(
    osp.join(root_dir, "data/preprocessed_data/preprocessed_data_train_2.pth")
)


In [ ]:
output_dir = None
#####
db = train_db
time_total = 0
mot_accums = []
results_seq = {}
for seq in val_sequences:
    # break
    tracker.reset()
    now = time.time()

    print(f"Tracking: {seq}")

    # data_loader = DataLoader(seq, batch_size=1, shuffle=False)
    with torch.no_grad():
        # for i, frame in enumerate(tqdm(data_loader)):
        for frame in db[str(seq)]:
            tracker.step(frame)

    results = tracker.get_results()
    results_seq[str(seq)] = results

    if seq.no_gt:
        print("No GT evaluation data available.")
    else:
        mot_accums.append(get_mot_accum(results, seq))

    time_total += time.time() - now

    print(f"Tracks found: {len(results)}")
    print(f"Runtime for {seq}: {time.time() - now:.1f} s.")

    if output_dir is not None:
        os.makedirs(output_dir, exist_ok=True)
        seq.write_results(results, os.path.join(output_dir))

print(f"Runtime for all sequences: {time_total:.1f} s.")
if mot_accums:
    evaluate_mot_accums(
        mot_accums,
        [str(s) for s in val_sequences if not s.no_gt],
        generate_overall=True,
    )


In [ ]:
plot_sequence(
    results_seq["MOT16-02"],
    [s for s in val_sequences if str(s) == "MOT16-02"][0],
    first_n_frames=2,
)


# create gif

In [ ]:
seq_name = "MOT16-02"
sequence = [s for s in val_sequences if str(s) == seq_name][0]
tracker_seq_res = results_seq[seq_name]


In [ ]:
frames = collect_frames_for_gif(sequence, tracker_seq_res)


In [ ]:
gif.save(frames, "MOT16-02_result_0001sec.gif", duration=0.001)
